In [ ]:
import geopandas
import rioxarray
import pathlib
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
slope_file = pathlib.Path(r"C:\Users\pearsonra\Documents\data\river_bathemetry\rec_domains\rec2_4\rec2_wshed_slopesV4.shp")
slope_data = geopandas.read_file(slope_file)
watershed_file = pathlib.Path(r"C:\Users\pearsonra\Documents\data\network\REC2_V4_FINAL\recV2_4_watersheds.shp")
watershed_data = geopandas.read_file(watershed_file)

## Data sources

* Building polygons - all sources - https://data.linz.govt.nz/layer/101292-nz-building-outlines-all-sources/
* Populations - 2013 - https://datafinder.stats.govt.nz/layer/8437-population-by-meshblock-2013-census/
* Building points - not many included - https://data.linz.govt.nz/layer/50245-nz-building-points-topo-150k/

### Catchment

In [ ]:
object_id = 253624
catchment_data = watershed_data[watershed_data['OBJECTID_1']==object_id]

In [ ]:
owslib.wms

### Load in DEMs
Note load in the DEM in tiles and calculate the slope at each pixel except on the tile edges. 
In future:
* We could clip offshore to save computation
* We could load at tiles to eliminate edge effects

In [ ]:
dem_14 = rioxarray.rioxarray.open_rasterio(r"C:\Users\pearsonra\Documents\data\DEMs\BackgroundDEM\NZDEM_SoS_15m\kx-14-palmerston-north-15m-dem-nzsosdem-v10-GTiff\NZDEM_SoS_v1-0_14_Palmerston North_gf.tif") # , chunks=True
dem_16 = rioxarray.rioxarray.open_rasterio(r"C:\Users\pearsonra\Documents\data\DEMs\BackgroundDEM\NZDEM_SoS_15m\kx-16-wellington-15m-dem-nzsosdem-v10-GTiff\NZDEM_SoS_v1-0_16_Wellington_gf.tif", chunks=True)

In [ ]:
catchment_data.geometry.bounds

In [ ]:
#dem_14=dem_14.rio.clip(catchment_data.geometry)
#dem_16=dem_16.rio.clip(catchment_data.geometry)

Overlap between the two files - can use either

In [ ]:
dem_14.plot()
#dem_16.plot()

### Previous slopes

In [ ]:
slope_data.head(3)

In [ ]:
f, ax = matplotlib.pyplot.subplots(1, 4, figsize=(30, 5))
slope_data[['MIN']].plot(ax=ax[0])
slope_data[['MAX']].plot(ax=ax[1])
slope_data[['STD']].plot(ax=ax[2])
slope_data[['MEAN']].plot(ax=ax[3])

### Estimate slope
Using the _planar method_ described in the arcGIS (How Slope works)[https://pro.arcgis.com/en/pro-app/2.8/tool-reference/spatial-analyst/how-slope-works.htm] page.

In [ ]:
dem_14.data

In [ ]:
slope = numpy.ones_like(dem_14.data) * numpy.nan

# get x, and y indices to calculate the slope at (i.e.)

### Load in all of the N/S island

In [ ]:
ni_dem = rioxarray.rioxarray.open_rasterio(r"C:\Users\pearsonra\Documents\data\watersheds\dems\nidem_from_tiff.nc", chunks=True)

In [ ]:
ni_dem.rio.reproject(2193)

In [ ]:
ni_dem.rio.clip(catchment_data.geometry)

In [ ]:
t.compute()

In [ ]:
t.plot(vmin=-2, vmax=1)

### Try to download tiles from WMTS

In [ ]:
import owslib
import owslib.wms
import owslib.wmts
import geoapis

In [ ]:
layer = 48081
apikey = "ENTER HERE"

In [ ]:
# https://lris.scinfo.org.nz/services;key=607f4a89db24407c94ac1df6274a523b/wmts/1.0.0/layer/48081/WMTSCapabilities.xml
# http://tiles-a.koordinates.com/services;key=KEY/tiles/v4/layer=48081/EPSG:3857/{z}/{x}/{y}.png
wms = owslib.wmts.WebMapTileService('https://lris.scinfo.org.nz/services;key=KEY/wmts/1.0.0/layer/48081')